In [1]:
import numpy as np
import cv2
from fractions import Fraction
from sympy import diff, Symbol
import sympy
import numpy as np
import math
from sympy import *
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist

        
# construct the argument parse and parse the arguments
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5, 5))
# load the image
image = cv2.imread('Long.png')
#image = cv2.resize(image, (1000, 750))
#cv2.imshow("images", image)
# define the list of boundaries
thres = 40
boundaries = [
	([0, 0, 230], [50, 50, 255]),#red
    ([0, 100, 230], [30, 180, 255]),#orange
    ([230, 0, 0], [255, 50,50 ]) # blue    
]
clone = image.copy()
# loop over the boundaries
ax=[]
ay=[]
for (lower, upper) in boundaries:
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
	# find the colors within the specified boundaries and apply
	# the mas
    mask = cv2.inRange(image, lower, upper)
    output1 = cv2.bitwise_and(image, image, mask = mask)
    x = output1.nonzero()
    ax.append(x[1][0])
    ay.append(x[0][0])
x_center=[]
y_center=[]
for i in ax:
    x_center.append(i-2000)
for i in ay:
    y_center.append(i-1500)
print("x_center",x_center)
print("y_center",y_center)
f=4.73#mm
sensor_size=[6.4,4.8]#mm
X1,Y1,Z1=0,0,0#A
X2,Y2,Z2=27.5,5.8,0#B cm
ratio=Distance(X1,Y1,Z1,X2,Y2,Z2)/(Distance(x_center[0],y_center[0],0,x_center[1],y_center[1],0)*1.6*0.0001)
print("ratio",int(ratio))
S2real=f*0.1*ratio#cm
for i in range(5):
    S = symbols('S')
    S=solve((1/S)+(1/S2real)-(1/f),S)[0]
    print("S",S)
    S2real = symbols('S2real')
    S2real=solve((1/S)+(1/S2real)-(1/f),S2real)[0]
    print("S2real",S2real)    

x_center [-313, 477, 270]
y_center [342, 211, 607]
ratio 219
S 4.95593291616156
S2real 103.754526306746
S 4.95593291616157
S2real 103.754526306742
S 4.95593291616157
S2real 103.754526306742
S 4.95593291616157
S2real 103.754526306742
S 4.95593291616157
S2real 103.754526306742


In [9]:
def Distance_o(x1,y1,S2real):
    dist = sympy.sqrt((sympy.sqrt((x1**2+y1**2))*ratio*0.1)**2+S2real**2)
    return dist
camera_real_x,camera_real_y,camera_real_z=40,20,S2real

X3,Y3,Z3=21,-8.3,0#cm
x1,y1=x_center[0],y_center[0]#pixel
x2,y2=x_center[1],y_center[1]
x3,y3=x_center[2],y_center[2]

A = Symbol('A',real=True)
B = Symbol('B',real=True)
C = Symbol('C',real=True)
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1*ratio*0.1-A,y1*ratio*0.1-B,(S+S2real)-C),(x2*ratio*0.1-A,y2*ratio*0.1-B,(S+S2real)-C))/(Distance_o(x1,y1,S2real)*Distance_o(x2,y2,S2real)))
#angle3=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X3-A,Y3-B,Z3-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X3,Y3,Z3)))
#angle4=sympy.acos(np.dot((x1*ratio*0.1-A,y1*ratio*0.1-B,0-C),(x3*ratio*0.1-A,y3*ratio*0.1-B,0-C))/(Distance_o(x1,y1,S2img)*Distance_o(x3,y3,S2img)))

delt_A=diff(angle1-angle2, A)
delt_B=diff(angle1-angle2, B)
delt_C=diff(angle1-angle2, C)

#delt_A_2=diff(angle3-angle4, A)
#delt_B_2=diff(angle3-angle4, B)
#delt_C_2=diff(angle3-angle4, C)

for i in range(500):
    delt_A=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_A_2=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B_2=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C_2=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    camera_real_x=camera_real_x-delt_A
    camera_real_y=camera_real_y-delt_B
    camera_real_z=camera_real_z-delt_C
    #print(delt_A)
    #print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)
    angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
    #angle2=sympy.acos(np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)))
    angle2=sympy.acos(np.dot((x1*ratio*0.1-A,y1*ratio*0.1-B,(S+S2real)-C),(x2*ratio*0.1-A,y2*ratio*0.1-B,(S+S2real)-C))/(Distance_o(x1,y1,S2real)*Distance_o(x2,y2,S2real)))
    print("x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
    #print(int(angle1-angle2))
    #print("angle1=%f angle2=%f"%(angle1,angle2))
    #if Abs(angle1-angle2)<0.01:
        #print("converge x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
        #break
    
        
    
    

x=40.001168 y=20.000574 z=103.756541
x=40.002335 y=20.001148 z=103.758555
x=40.003503 y=20.001721 z=103.760570
x=40.004671 y=20.002295 z=103.762584
x=40.005838 y=20.002869 z=103.764599
x=40.007006 y=20.003443 z=103.766613
x=40.008174 y=20.004017 z=103.768628
x=40.009342 y=20.004591 z=103.770642
x=40.010509 y=20.005164 z=103.772657
x=40.011677 y=20.005738 z=103.774672
x=40.012845 y=20.006312 z=103.776686
x=40.014012 y=20.006886 z=103.778701
x=40.015180 y=20.007460 z=103.780715
x=40.016348 y=20.008033 z=103.782730
x=40.017515 y=20.008607 z=103.784744
x=40.018683 y=20.009181 z=103.786759
x=40.019851 y=20.009755 z=103.788773
x=40.021019 y=20.010329 z=103.790788
x=40.022186 y=20.010902 z=103.792802
x=40.023354 y=20.011476 z=103.794817
x=40.024522 y=20.012050 z=103.796831
x=40.025689 y=20.012624 z=103.798846
x=40.026857 y=20.013198 z=103.800860
x=40.028025 y=20.013772 z=103.802875
x=40.029192 y=20.014345 z=103.804889
x=40.030360 y=20.014919 z=103.806904
x=40.031528 y=20.015493 z=103.808918
x

x=40.260397 y=20.127961 z=104.203764
x=40.261565 y=20.128535 z=104.205779
x=40.262732 y=20.129108 z=104.207793
x=40.263900 y=20.129682 z=104.209808
x=40.265068 y=20.130256 z=104.211822
x=40.266235 y=20.130830 z=104.213837
x=40.267403 y=20.131404 z=104.215851
x=40.268571 y=20.131977 z=104.217866
x=40.269738 y=20.132551 z=104.219880
x=40.270906 y=20.133125 z=104.221895
x=40.272074 y=20.133699 z=104.223910
x=40.273242 y=20.134273 z=104.225924
x=40.274409 y=20.134846 z=104.227939
x=40.275577 y=20.135420 z=104.229953
x=40.276745 y=20.135994 z=104.231968
x=40.277912 y=20.136568 z=104.233982
x=40.279080 y=20.137142 z=104.235997
x=40.280248 y=20.137716 z=104.238011
x=40.281415 y=20.138289 z=104.240026
x=40.282583 y=20.138863 z=104.242040
x=40.283751 y=20.139437 z=104.244055
x=40.284919 y=20.140011 z=104.246069
x=40.286086 y=20.140585 z=104.248084
x=40.287254 y=20.141158 z=104.250098
x=40.288422 y=20.141732 z=104.252113
x=40.289589 y=20.142306 z=104.254127
x=40.290757 y=20.142880 z=104.256142
x

x=40.519626 y=20.255348 z=104.650988
x=40.520794 y=20.255921 z=104.653002
x=40.521961 y=20.256495 z=104.655017
x=40.523129 y=20.257069 z=104.657031
x=40.524297 y=20.257643 z=104.659046
x=40.525464 y=20.258217 z=104.661060
x=40.526632 y=20.258791 z=104.663075
x=40.527800 y=20.259364 z=104.665089
x=40.528968 y=20.259938 z=104.667104
x=40.530135 y=20.260512 z=104.669119
x=40.531303 y=20.261086 z=104.671133
x=40.532471 y=20.261660 z=104.673148
x=40.533638 y=20.262233 z=104.675162
x=40.534806 y=20.262807 z=104.677177
x=40.535974 y=20.263381 z=104.679191
x=40.537141 y=20.263955 z=104.681206
x=40.538309 y=20.264529 z=104.683220
x=40.539477 y=20.265102 z=104.685235
x=40.540645 y=20.265676 z=104.687249
x=40.541812 y=20.266250 z=104.689264
x=40.542980 y=20.266824 z=104.691278
x=40.544148 y=20.267398 z=104.693293
x=40.545315 y=20.267972 z=104.695307
x=40.546483 y=20.268545 z=104.697322
x=40.547651 y=20.269119 z=104.699336
x=40.548818 y=20.269693 z=104.701351
x=40.549986 y=20.270267 z=104.703365
x

In [ ]:
camera_real_x,camera_real_y,camera_real_z=56.211932,15.395214,3.000000
X1,Y1,Z1=100,100,33
X2,Y2,Z2=200,200,33
x1,y1=10,10
x2,y2=20,20
angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)))
print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)

print(angle1-angle2)

In [ ]:
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist
A = Symbol('A')
B = Symbol('B')
C = Symbol('C')

X1,Y1,Z1=100,100,30
X2,Y2,Z2=100,(100+30/sympy.sqrt(3)),30
x1,y1=1,1
x2,y2=2,2


angle1=np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)
angle2=np.dot((x1-A,y1-B,0-C),(x2-A,x2-B,0-C))/Distance(A,B,C,x1,y1,0)*Distance(A,B,C,x2,y2,0)


a=diff(angle1-angle2, A).subs({A:5,B:10,C:3})
print(a+5)
camera_real_x,camera_real_y,camera_real_z=100,100,0

angle1=np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)
angle2=np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)
#print(type(angle1))
print("angle1=%f angle2=%f"%(angle1,angle2))

abs(angle1-angle2)<1
X = Symbol('X',real=True)

diff(sympy.Abs(X**2+2), X).subs({X:5}).n(10)

In [ ]:
X1,Y1,Z1=0,0,30
X2,Y2,Z2=0,(30/sympy.sqrt(3)),0
A,B,C=0,0,0
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2))
print("angle1=%f"% angle1)
angle1<1
print(angle1.n(10))
sympy.acos(0.2)

In [ ]:
X1,Y1,Z1=0,0,30
X2,Y2,Z2=0,0,0
A,B,C=0,(30/sympy.sqrt(3)),0
angle1=np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2))
print(Distance(A,B,C,X1,Y1,Z1),Distance(A,B,C,X2,Y2,Z2))
print(angle1)
sympy.acos(angle1)